In [1]:
import numpy as np
import torch
import optuna
from deepchem.feat import GraphData
import pandas as pd
from deepchem.feat import DMPNNFeaturizer
import deepchem as dc
import deepchem as dc
from deepchem.models import MPNNModel
from deepchem.data import NumpyDataset
from deepchem.splits import RandomStratifiedSplitter
from deepchem.feat.molecule_featurizers import DMPNNFeaturizer
from deepchem.data.data_loader import CSVLoader
from deepchem.metrics import Metric
from sklearn.metrics import roc_auc_score
from deepchem.models.optimizers import ExponentialDecay
import warnings
import multiprocessing
import optuna
#from tensorflow.keras.optimizers.schedules import ExponentialDecay
from deepchem.splits import RandomStratifiedSplitter
from deepchem.metrics import Metric, roc_auc_score
from deepchem.data import DiskDataset
import multiprocessing
import torch
from deepchem.data import DiskDataset
import os
import deepchem as dc
from openpom.feat.graph_featurizer import GraphFeaturizer, GraphConvConstants
from openpom.utils.data_utils import get_class_imbalance_ratio

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\__init__.py)


In [2]:
def get_graphs(weighted_adj_matrix):
    node_features = weighted_adj_matrix.diagonal().reshape(-1, 1)
    if not node_features.flags.writeable:
        node_features = node_features.copy()
    edges = np.array(np.nonzero(weighted_adj_matrix))
    edge_weights = weighted_adj_matrix[edges.T[:, 0], edges.T[:, 1]]
    graph = GraphData(node_features=node_features, edge_index=edges, edge_features=edge_weights.reshape(-1, 1))
    return graph

In [3]:
TASKS = [
'alcoholic', 'aldehydic', 'alliaceous', 'almond', 'amber', 'animal',
'anisic', 'apple', 'apricot', 'aromatic', 'balsamic', 'banana', 'beefy',
'bergamot', 'berry', 'bitter', 'black currant', 'brandy', 'burnt',
'buttery', 'cabbage', 'camphoreous', 'caramellic', 'cedar', 'celery',
'chamomile', 'cheesy', 'cherry', 'chocolate', 'cinnamon', 'citrus', 'clean',
'clove', 'cocoa', 'coconut', 'coffee', 'cognac', 'cooked', 'cooling',
'cortex', 'coumarinic', 'creamy', 'cucumber', 'dairy', 'dry', 'earthy',
'ethereal', 'fatty', 'fermented', 'fishy', 'floral', 'fresh', 'fruit skin',
'fruity', 'garlic', 'gassy', 'geranium', 'grape', 'grapefruit', 'grassy',
'green', 'hawthorn', 'hay', 'hazelnut', 'herbal', 'honey', 'hyacinth',
'jasmin', 'juicy', 'ketonic', 'lactonic', 'lavender', 'leafy', 'leathery',
'lemon', 'lily', 'malty', 'meaty', 'medicinal', 'melon', 'metallic',
'milky', 'mint', 'muguet', 'mushroom', 'musk', 'musty', 'natural', 'nutty',
'odorless', 'oily', 'onion', 'orange', 'orangeflower', 'orris', 'ozone',
'peach', 'pear', 'phenolic', 'pine', 'pineapple', 'plum', 'popcorn',
'potato', 'powdery', 'pungent', 'radish', 'raspberry', 'ripe', 'roasted',
'rose', 'rummy', 'sandalwood', 'savory', 'sharp', 'smoky', 'soapy',
'solvent', 'sour', 'spicy', 'strawberry', 'sulfurous', 'sweaty', 'sweet',
'tea', 'terpenic', 'tobacco', 'tomato', 'tropical', 'vanilla', 'vegetable',
'vetiver', 'violet', 'warm', 'waxy', 'weedy', 'winey', 'woody'
]
print("No of tasks: ", len(TASKS))

No of tasks:  138


In [4]:
from deepchem.feat import DMPNNFeaturizer
import deepchem as dc


featurizer = DMPNNFeaturizer()

smiles_field = 'nonStereoSMILES'


loader = dc.data.CSVLoader(
    tasks=TASKS,  
    feature_field=smiles_field,
    featurizer=featurizer
)

In [5]:

dataset = loader.create_dataset(inputs=["DMPNNpruned_without hydrogen_curated_GS_LF_merged_4812_QM_cleaned.csv"])
n_tasks = len(dataset.tasks)
dataset.X
len(dataset.X)


4812

In [6]:
data = np.load("DMPNNpruned_graph_data_cleaned.npz",allow_pickle=True)
mtx_list = data['mtx']
qm_graphs = []
for mtx in mtx_list:
    qm_graphs.append(get_graphs(mtx))
QM_X = np.asarray(qm_graphs)
QM_X
len(QM_X)

4812

In [7]:
X = np.asarray([i for i in zip(dataset.X, QM_X)])
new_dataset = dc.data.NumpyDataset(X=X, y=dataset.y, n_tasks=138, ids=dataset.ids)

In [8]:
new_dataset

<NumpyDataset X.shape: (4812, 2), y.shape: (4812, 138), w.shape: (4812, 1), task_names: [  0   1   2 ... 135 136 137]>

In [9]:
# get train valid test splits

randomstratifiedsplitter = dc.splits.RandomStratifiedSplitter()
train_dataset, test_dataset, valid_dataset = randomstratifiedsplitter.train_valid_test_split(new_dataset, frac_train = 0.8, frac_valid = 0.1, frac_test = 0.1, seed = 1)

In [10]:
def preprocess_dataset(dataset):
    for i in range(len(dataset.X)):
        if isinstance(dataset.X[i], GraphData):
            dataset.X[i].node_features = np.array(dataset.X[i].node_features, copy=True)
    return dataset

# Then in your objective function, before model creation:
train_dataset = preprocess_dataset(train_dataset)
valid_dataset = preprocess_dataset(valid_dataset)

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Tuple, Union, Optional, Callable, Dict

from deepchem.models.losses import Loss, L2Loss
from deepchem.models.torch_models.torch_model import TorchModel
from deepchem.models.optimizers import Optimizer, LearningRateSchedule

from openpom.layers.pom_ffn import CustomPositionwiseFeedForward
from openpom.utils.loss import CustomMultiLabelLoss
from openpom.utils.optimizer import get_optimizer

try:
    import dgl
    from dgl import DGLGraph
    from dgl.nn.pytorch import Set2Set
    from openpom.layers.pom_mpnn_gnn import CustomMPNNGNN
except (ImportError, ModuleNotFoundError):
    raise ImportError('This module requires dgl and dgllife')


class MPNNPOM(nn.Module):

    def __init__(self,
                 n_tasks: int,
                 node_out_feats: int = 64,
                 edge_hidden_feats: int = 128,
                 edge_out_feats: int = 64,
                 num_step_message_passing: int = 3,
                 mpnn_residual: bool = True,
                 message_aggregator_type: str = 'sum',
                 mode: str = 'classification',
                 number_atom_features: int = 133,
                 number_bond_features: int = 14,
                 n_classes: int = 1,
                 nfeat_name: str = 'x',
                 efeat_name: str = 'edge_attr',
                 readout_type: str = 'set2set',
                 num_step_set2set: int = 6,
                 num_layer_set2set: int = 3,
                 ffn_hidden_list: List = [300],
                 ffn_embeddings: int = 256,
                 ffn_activation: str = 'relu',
                 ffn_dropout_p: float = 0.0,
                 ffn_dropout_at_input_no_act: bool = True):

        if mode not in ['classification', 'regression']:
            raise ValueError(
                "mode must be either 'classification' or 'regression'")

        super(MPNNPOM, self).__init__()

        self.n_tasks: int = n_tasks
        self.mode: str = mode
        self.n_classes: int = n_classes
        self.nfeat_name: str = nfeat_name
        self.efeat_name: str = efeat_name
        self.readout_type: str = readout_type
        self.ffn_embeddings: int = ffn_embeddings
        self.ffn_activation: str = ffn_activation
        self.ffn_dropout_p: float = ffn_dropout_p

        if mode == 'classification':
            self.ffn_output: int = n_tasks * n_classes
        else:
            self.ffn_output = n_tasks

        self.mpnn: nn.Module = CustomMPNNGNN(
            node_in_feats=number_atom_features,
            node_out_feats=node_out_feats,
            edge_in_feats=number_bond_features,
            edge_hidden_feats=edge_hidden_feats,
            num_step_message_passing=num_step_message_passing,
            residual=mpnn_residual,
            message_aggregator_type=message_aggregator_type)
        
        QM_node_out_features = 50
        
        self.QM_mpnn: nn.Module = CustomMPNNGNN(
            node_in_feats=1,
            node_out_feats=QM_node_out_features,
            edge_in_feats=1,
            edge_hidden_feats=16,
            num_step_message_passing=2,
            residual=True,
            message_aggregator_type='sum')

        self.project_edge_feats: nn.Module = nn.Sequential(
            nn.Linear(number_bond_features, edge_out_feats), nn.ReLU())

        if self.readout_type == 'set2set':
            self.readout_set2set: nn.Module = Set2Set(
                input_dim=node_out_feats + edge_out_feats,
                n_iters=num_step_set2set,
                n_layers=num_layer_set2set)
            ffn_input: int = 2 * (node_out_feats + edge_out_feats)
        elif self.readout_type == 'global_sum_pooling':
            ffn_input = node_out_feats + edge_out_feats
        else:
            raise Exception("readout_type invalid")

        if ffn_embeddings is not None:
            d_hidden_list: List = ffn_hidden_list + [ffn_embeddings]

        self.ffn: nn.Module = CustomPositionwiseFeedForward(
            d_input=ffn_input + QM_node_out_features,
            # d_input=ffn_input,
            d_hidden_list=d_hidden_list,
            d_output=self.ffn_output,
            activation=ffn_activation,
            dropout_p=ffn_dropout_p,
            dropout_at_input_no_act=ffn_dropout_at_input_no_act)

    def _readout(self, g: DGLGraph, node_encodings: torch.Tensor,
                 edge_feats: torch.Tensor) -> torch.Tensor:

        g.ndata['node_emb'] = node_encodings
        g.edata['edge_emb'] = self.project_edge_feats(edge_feats)

        def message_func(edges) -> Dict:
            """
            The message function to generate messages
            along the edges for DGLGraph.send_and_recv()
            """
            src_msg: torch.Tensor = torch.cat(
                (edges.src['node_emb'], edges.data['edge_emb']), dim=1)
            return {'src_msg': src_msg}

        def reduce_func(nodes) -> Dict:
            """
            The reduce function to aggregate the messages
            for DGLGraph.send_and_recv()
            """
            src_msg_sum: torch.Tensor = torch.sum(nodes.mailbox['src_msg'],
                                                  dim=1)
            return {'src_msg_sum': src_msg_sum}

        # radius 0 combination to fold atom and bond embeddings together
        g.send_and_recv(g.edges(),
                        message_func=message_func,
                        reduce_func=reduce_func)

        if self.readout_type == 'set2set':
            batch_mol_hidden_states: torch.Tensor = self.readout_set2set(
                g, g.ndata['src_msg_sum'])
        elif self.readout_type == 'global_sum_pooling':
            batch_mol_hidden_states = dgl.sum_nodes(g, 'src_msg_sum')

        # batch_size x (node_out_feats + edge_out_feats)
        return batch_mol_hidden_states

    def forward(
        self, graphs: tuple[DGLGraph]
    ) -> Union[tuple[torch.Tensor, torch.Tensor, torch.Tensor], torch.Tensor]:
        g, qm_g = graphs
        node_feats: torch.Tensor = g.ndata[self.nfeat_name]
        edge_feats: torch.Tensor = g.edata[self.efeat_name]

        qm_node_feats: torch.Tensor = qm_g.ndata[self.nfeat_name]
        qm_edge_feats: torch.Tensor = qm_g.edata[self.efeat_name]

        node_encodings: torch.Tensor = self.mpnn(g, node_feats, edge_feats)

        QM_encodings: torch.Tensor = self.QM_mpnn(qm_g, qm_node_feats, qm_edge_feats)
        qm_g.ndata['node_emb'] = QM_encodings
        molecular_QM_encodings = dgl.sum_nodes(qm_g, 'node_emb')
        molecular_encodings: torch.Tensor = self._readout(
            g, node_encodings, edge_feats)
        if self.readout_type == 'global_sum_pooling':
            molecular_encodings = F.softmax(molecular_encodings, dim=1)

        embeddings: torch.Tensor
        out: torch.Tensor
        embeddings, out = self.ffn(torch.concat((molecular_encodings, molecular_QM_encodings), dim=1))
        #embeddings, out = self.ffn(molecular_QM_encodings)
        # embeddings, out = self.ffn(molecular_encodings)

        if self.mode == 'classification':
            if self.n_tasks == 1:
                logits: torch.Tensor = out.view(-1, self.n_classes)
            else:
                logits = out.view(-1, self.n_tasks, self.n_classes)
            proba: torch.Tensor = F.sigmoid(
                logits)  # (batch, n_tasks, classes)
            if self.n_classes == 1:
                proba = proba.squeeze(-1)  # (batch, n_tasks)
            return proba, logits, embeddings
        else:
            return out


class MPNNPOMModel(TorchModel):

    def __init__(self,
                 n_tasks: int,
                 class_imbalance_ratio: Optional[List] = None,
                 loss_aggr_type: str = 'sum',
                 learning_rate: Union[float, LearningRateSchedule] = 0.001,
                 batch_size: int = 100,
                 node_out_feats: int = 64,
                 edge_hidden_feats: int = 128,
                 edge_out_feats: int = 64,
                 num_step_message_passing: int = 3,
                 mpnn_residual: bool = True,
                 message_aggregator_type: str = 'sum',
                 mode: str = 'regression',
                 number_atom_features: int = 133,
                 number_bond_features: int = 14,
                 n_classes: int = 1,
                 readout_type: str = 'set2set',
                 num_step_set2set: int = 6,
                 num_layer_set2set: int = 3,
                 ffn_hidden_list: List = [300],
                 ffn_embeddings: int = 256,
                 ffn_activation: str = 'relu',
                 ffn_dropout_p: float = 0.0,
                 ffn_dropout_at_input_no_act: bool = True,
                 weight_decay: float = 1e-5,
                 self_loop: bool = False,
                 optimizer_name: str = 'adam',
                 device_name: Optional[str] = None,
                 **kwargs):
        model: nn.Module = MPNNPOM(
            n_tasks=n_tasks,
            node_out_feats=node_out_feats,
            edge_hidden_feats=edge_hidden_feats,
            edge_out_feats=edge_out_feats,
            num_step_message_passing=num_step_message_passing,
            mpnn_residual=mpnn_residual,
            message_aggregator_type=message_aggregator_type,
            mode=mode,
            number_atom_features=number_atom_features,
            number_bond_features=number_bond_features,
            n_classes=n_classes,
            readout_type=readout_type,
            num_step_set2set=num_step_set2set,
            num_layer_set2set=num_layer_set2set,
            ffn_hidden_list=ffn_hidden_list,
            ffn_embeddings=ffn_embeddings,
            ffn_activation=ffn_activation,
            ffn_dropout_p=ffn_dropout_p,
            ffn_dropout_at_input_no_act=ffn_dropout_at_input_no_act)

        if class_imbalance_ratio and (len(class_imbalance_ratio) != n_tasks):
            raise Exception("size of class_imbalance_ratio \
                            should be equal to n_tasks")

        if mode == 'regression':
            loss: Loss = L2Loss()
            output_types: List = ['prediction']
        else:
            loss = CustomMultiLabelLoss(
                class_imbalance_ratio=class_imbalance_ratio,
                loss_aggr_type=loss_aggr_type,
                device=device_name)
            output_types = ['prediction', 'loss', 'embedding']

        optimizer: Optimizer = get_optimizer(optimizer_name)
        optimizer.learning_rate = learning_rate
        if device_name is not None:
            device: Optional[torch.device] = torch.device(device_name)
        else:
            device = None
        super(MPNNPOMModel, self).__init__(model,
                                           loss=loss,
                                           output_types=output_types,
                                           optimizer=optimizer,
                                           learning_rate=learning_rate,
                                           batch_size=batch_size,
                                           device=device,
                                           **kwargs)

        self.weight_decay: float = weight_decay
        self._self_loop: bool = self_loop
        self.regularization_loss: Callable = self._regularization_loss

    def _regularization_loss(self) -> torch.Tensor:
        l1_regularization: torch.Tensor = torch.tensor(0., requires_grad=True)
        l2_regularization: torch.Tensor = torch.tensor(0., requires_grad=True)
        for name, param in self.model.named_parameters():
            if 'bias' not in name:
                l1_regularization = l1_regularization + torch.norm(param, p=1)
                l2_regularization = l2_regularization + torch.norm(param, p=2)
        l1_norm: torch.Tensor = self.weight_decay * l1_regularization
        l2_norm: torch.Tensor = self.weight_decay * l2_regularization
        return l1_norm + l2_norm

    def _prepare_batch(
        self, batch: Tuple[List, List, List]
    ) -> Tuple[DGLGraph, List[torch.Tensor], List[torch.Tensor]]:
        inputs: List
        labels: List
        weights: List

        inputs, labels, weights = batch
        dgl_graphs1: List[DGLGraph] = [
            graphs[0].to_dgl_graph(self_loop=self._self_loop)
            for graphs in inputs[0]
        ]
        g1: DGLGraph = dgl.batch(dgl_graphs1).to(self.device)

        dgl_graphs2: List[DGLGraph] = [
            graphs[1].to_dgl_graph(self_loop=self._self_loop)
            for graphs in inputs[0]
        ]
        g2: DGLGraph = dgl.batch(dgl_graphs2).to(self.device)
        _, labels, weights = super(MPNNPOMModel, self)._prepare_batch(
            ([], labels, weights))
        return (g1, g2), labels, weights


In [12]:
def convert_to_disk_dataset(train_dataset, output_dir):
    """
    Converts a deepchem.data.datasets.NumpyDataset to a deepchem.data.datasets.DiskDataset.
    
    Parameters:
    train_dataset (deepchem.data.datasets.NumpyDataset): The input NumpyDataset to convert.
    output_dir (str): The directory to save the DiskDataset.
    
    Returns:
    deepchem.data.datasets.DiskDataset: The converted DiskDataset.
    """
    os.makedirs(output_dir, exist_ok=True)
    
    X, y, w, ids = train_dataset.X, train_dataset.y, train_dataset.w, train_dataset.ids
    
    disk_dataset = dc.data.DiskDataset.from_numpy(X, y, w, ids, data_dir=output_dir)
    
    return disk_dataset
# output_dir = 'C:/Users/lenovo/Documents/QM_work/train_dataset'
output_dir_1 = r'C:\Users\sarab\OneDrive\Desktop\hyperparamter_dmpnn\train_dataset'


train_dataset = convert_to_disk_dataset(train_dataset, output_dir_1)
# output_dir = 'C:/Users/lenovo/Documents/QM_work/train_dataset'
output_dir_2 = r'C:\Users\sarab\OneDrive\Desktop\hyperparamter_dmpnn\valid_dataset'
valid_dataset = convert_to_disk_dataset(valid_dataset, output_dir_2)

In [13]:
train_ratios = get_class_imbalance_ratio(train_dataset)
assert len(train_ratios) == n_tasks

In [14]:
import optuna
import deepchem as dc
from deepchem.models.optimizers import ExponentialDecay
import numpy as np
from multiprocessing import Pool
import logging
import os

# Basic logging without timestamp
class CustomFormatter(logging.Formatter):
    def format(self, record):
        return record.getMessage()

logger = logging.getLogger(__name__)
handler = logging.StreamHandler()
handler.setFormatter(CustomFormatter())
logger.handlers = [handler]
logger.setLevel(logging.INFO)

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0
        return self.early_stop

def objective(trial):
    logger.info(f"\nTrial {trial.number}:")
    params = {
        "initial_lr_rate": trial.suggest_float("initial_lr_rate", 1e-4, 1e-2, log=True),
        "decay_rate": trial.suggest_float("decay_rate", 0.1, 0.9),
        "decay_steps": trial.suggest_categorical("decay_steps", [764, 384, 96]),
        "batch_size": trial.suggest_categorical("batch_size", [16, 25, 32]),
        "loss_aggr_type": trial.suggest_categorical("loss_aggr_type", ["sum", "mean"]),
        "num_step_message_passing": trial.suggest_int("num_step_message_passing", 2, 5),
        "message_aggregator_type": trial.suggest_categorical("message_aggregator_type", ["sum", "mean", "max"]),
        "ffn_hidden_list": trial.suggest_categorical("ffn_hidden_list", ["(256, 256)", "(392, 392)", "(512, 512)"]),
        "ffn_dropout_p": trial.suggest_float("ffn_dropout_p", 0.0, 0.5),
        "ffn_dropout_at_input_no_act": trial.suggest_categorical("ffn_dropout_at_input_no_act", [True, False]),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True)
    }
    trial_info = f"\nTrial {trial.number}:\n" + "\n".join([f"{key}: {value}" for key, value in params.items()])
    logger.info(trial_info)
    # for key, value in params.items():
    #     logger.info(f"{key}: {value}")

    learning_rate = ExponentialDecay(
        initial_rate=params["initial_lr_rate"],
        decay_rate=params["decay_rate"],
        decay_steps=params["decay_steps"],
        staircase=True
    )

    model = MPNNPOMModel(
    n_tasks=n_tasks,
    batch_size=params["batch_size"],
    learning_rate=learning_rate,
    class_imbalance_ratio=train_ratios,
    loss_aggr_type=params["loss_aggr_type"],
    node_out_feats=25,
    edge_hidden_feats=75,
    edge_out_feats=25,
    num_step_message_passing=params["num_step_message_passing"],
    mpnn_residual=True,
    message_aggregator_type=params["message_aggregator_type"],
    mode='classification',
    number_atom_features=133,
    number_bond_features=14,
    n_classes=1,
    readout_type='set2set',
    num_step_set2set=3,
    num_layer_set2set=2,
    ffn_hidden_list=list(eval(params["ffn_hidden_list"])),  # Convert to list explicitly
    ffn_embeddings=256,
    ffn_activation='relu',
    ffn_dropout_p=params["ffn_dropout_p"],
    ffn_dropout_at_input_no_act=params["ffn_dropout_at_input_no_act"],
    weight_decay=params["weight_decay"],
    self_loop=False,
    optimizer_name='adam',
    log_frequency=32,
    model_dir=f'./examples/experiments/trial_{trial.number}',
    device_name='cpu'
    )
    metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
    early_stopping = EarlyStopping(patience=5, min_delta=0.001)
    best_valid_score = 0
    
    for epoch in range(1, 60):
        try:
            train_loss = model.fit(train_dataset, nb_epoch=1, max_checkpoints_to_keep=1, 
                                 deterministic=False, restore=epoch>1)
            valid_scores = model.evaluate(valid_dataset, [metric])['roc_auc_score']
            
            logger.info(f"Trial {trial.number} Epoch {epoch} - Train Loss: {train_loss:.4f}, Valid Score: {valid_scores:.4f}")
            
            if valid_scores > best_valid_score:
                best_valid_score = valid_scores
            
            if early_stopping(train_loss):
                break
                
            trial.report(valid_scores, epoch)
            if trial.should_prune():
                raise optuna.TrialPruned()
                
        except Exception as e:
            logger.error(f"Trial {trial.number} failed: {str(e)}")
            raise optuna.TrialPruned()
    
    return best_valid_score

def optimize_hyperparameters(n_trials=100, n_jobs=4):
    study = optuna.create_study(
        direction="maximize",
        pruner=optuna.pruners.MedianPruner(),
        study_name="mpnn_optimization"
    )
    
    try:
        study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs, timeout=None)
        return study
    except Exception as e:
        logger.error(f"Optimization failed: {str(e)}")
        raise

import os

if __name__ == "__main__":
    os.makedirs('./examples/experiments', exist_ok=True)
    num_jobs = os.cpu_count()  # Get the number of available CPU cores
    print(f"Using {num_jobs} cores for optimization.")
    
    study = optimize_hyperparameters(n_trials=60, n_jobs=num_jobs)
    study.trials_dataframe().to_csv('optimization_results.csv')


[I 2025-01-02 13:09:41,846] A new study created in memory with name: mpnn_optimization

Trial 0:

Trial 1:

Trial 2:

Trial 3:

Trial 4:

Trial 0:
initial_lr_rate: 0.00011273372644699478
decay_rate: 0.1629674306653527
decay_steps: 96
batch_size: 32
loss_aggr_type: mean
num_step_message_passing: 2
message_aggregator_type: mean
ffn_hidden_list: (256, 256)
ffn_dropout_p: 0.19772718284878132
ffn_dropout_at_input_no_act: False
weight_decay: 7.190137230132782e-05

Trial 5:

Trial 6:

Trial 7:

Trial 8:

Trial 9:

Trial 10:

Trial 11:

Trial 12:

Trial 1:
initial_lr_rate: 0.0021452735649492996
decay_rate: 0.2801226587951189
decay_steps: 384
batch_size: 25
loss_aggr_type: mean
num_step_message_passing: 3
message_aggregator_type: max
ffn_hidden_list: (512, 512)
ffn_dropout_p: 0.05922283459955768
ffn_dropout_at_input_no_act: False
weight_decay: 5.655403650329639e-05

Trial 13:

Trial 14:

Trial 15:

Trial 2:
initial_lr_rate: 0.0026055073746078412
decay_rate: 0.7301431374142417
decay_steps: 96
ba

Using 16 cores for optimization.



Trial 3:
initial_lr_rate: 0.0002530516956518053
decay_rate: 0.15649401897559034
decay_steps: 384
batch_size: 16
loss_aggr_type: mean
num_step_message_passing: 5
message_aggregator_type: max
ffn_hidden_list: (392, 392)
ffn_dropout_p: 0.4332773981658581
ffn_dropout_at_input_no_act: True
weight_decay: 1.1117838162794856e-05

Trial 4:
initial_lr_rate: 0.002090709827057367
decay_rate: 0.46007681617696317
decay_steps: 384
batch_size: 16
loss_aggr_type: sum
num_step_message_passing: 3
message_aggregator_type: max
ffn_hidden_list: (512, 512)
ffn_dropout_p: 0.40365418402695463
ffn_dropout_at_input_no_act: True
weight_decay: 3.6945917677082526e-06

Trial 5:
initial_lr_rate: 0.00025076379099794963
decay_rate: 0.7754675569846277
decay_steps: 384
batch_size: 16
loss_aggr_type: mean
num_step_message_passing: 3
message_aggregator_type: sum
ffn_hidden_list: (256, 256)
ffn_dropout_p: 0.22156197222751245
ffn_dropout_at_input_no_act: False
weight_decay: 2.0194803675643545e-05

Trial 6:
initial_lr_rate: 